# Dataset Preprocessing

Now let's transform the dataset into the desired dataset.

Tasks:
- To balance the dataset
- To tokenize text
- To pad text
- To transform categories into numerical categories

In [1]:
from datasets import load_dataset

dataset = load_dataset("hackathon-pln-es/readability-es-caes")

e:\python_enviroments\envs\nlpenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = dataset["train"]

In [3]:
import pandas as pd
df = pd.DataFrame(train_data)
df

,category,text,level,level-3
0,B1,Hola Carlos ! Hace mucho que no te veo hombre ...,simple,intermediate
1,A2,"Hola querida Sara , espero que estes muy bien ...",simple,basic
2,A1,Como estais ? ! Hola amigos ! ? Espero que tod...,simple,basic
3,A2,"Cuando llegué a el destino , le propuso que si...",simple,basic
4,A1,¿ Que tal ? un beso Espero que todo va bien po...,simple,basic
...,...,...,...,...
31144,B1,Buenas tardes ! 29_de_mayo_de_2019 salamanca Q...,simple,intermediate
31145,B2,Tengo la seguridad de mí mismo para contener e...,complex,intermediate
31146,B1,"Distinguidos señor / señra : Hola , gerente . ...",simple,intermediate
31147,B2,Le escribo esta carta porque me gustaría inmen...,complex,intermediate


# Filtering

In [4]:
#casting text to String
df['text'] = df['text'].astype(str)

In [5]:
df = df[df['text'].apply(len) < 512]

In [6]:
df.shape

(7424, 4)

In [7]:
# Count the number of elements per category
category_counts = df["category"].value_counts()
category_counts

category
A1    4476
A2    1997
B1     787
B2     100
C1      64
Name: count, dtype: int64

In [8]:
# Count the number of elements per category
category_counts = df["category"].value_counts()
category_counts

category
A1    4476
A2    1997
B1     787
B2     100
C1      64
Name: count, dtype: int64

# Balancing

In [10]:
df_cleaned = df[['category', 'text']]
df_cleaned

,category,text
4,A1,¿ Que tal ? un beso Espero que todo va bien po...
5,A1,¿ Cómo estas chica ? Había regresado de México...
9,A1,"Mi padres , tengo el español clase , desde las..."
10,A1,¿ Cómo está ? Está muy bine . Ahora vivo en la...
13,A1,"Buenos dias , Me llamo Manon , soy francesa de..."
...,...,...
31123,A1,¿ que tal ? ¡ hola ! soy ikbal niyya una nueva...
31124,A2,None
31130,A2,Hola ! Mis últimas vacaciones estuvieron en in...
31133,A2,None


In [11]:
from imblearn.over_sampling import RandomOverSampler

# Create an oversampler
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Resample the dataset to balance the categories
X_resampled, y_resampled = oversampler.fit_resample(df_cleaned.drop("category", axis=1), df_cleaned["category"])



In [12]:
X_resampled

,text
0,¿ Que tal ? un beso Espero que todo va bien po...
1,¿ Cómo estas chica ? Había regresado de México...
2,"Mi padres , tengo el español clase , desde las..."
3,¿ Cómo está ? Está muy bine . Ahora vivo en la...
4,"Buenos dias , Me llamo Manon , soy francesa de..."
...,...
22375,"Abdellatif_Zaki calle_de_los_palmeros,n_70 , O..."
22376,"Compaña de gas y electricidad muy sr , mio c,_..."
22377,"Estimado senor Sibelgaz , me llamo AH y he rec..."
22378,Muy senores mios : les saludio atentamente . F...


In [13]:
max_length = X_resampled['text'].apply(len).max()
max_length

511

In [14]:
y_resampled

0        A1
1        A1
2        A1
3        A1
4        A1
         ..
22375    C1
22376    C1
22377    C1
22378    C1
22379    C1
Name: category, Length: 22380, dtype: object

In [15]:
# Count the number of elements per category
category_counts = y_resampled.value_counts()
category_counts

category
A1    4476
A2    4476
B1    4476
C1    4476
B2    4476
Name: count, dtype: int64

In [16]:
from sklearn.utils import shuffle

# Shuffle your balanced dataset
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)


In [17]:
# Assuming X_resampled and y_resampled contain your shuffled data
shuffled_df = pd.DataFrame({'text': X_resampled['text'], 'category': y_resampled})

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the DataFrame into training (80%) and validation (20%) sets
train_df, val_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)

In [19]:
# Count the number of elements per category
train_category_counts = train_df["category"].value_counts()
train_category_counts

category
C1    3640
A2    3575
B2    3563
B1    3563
A1    3563
Name: count, dtype: int64

In [20]:
# Count the number of elements per category
val_category_counts = val_df["category"].value_counts()
val_category_counts

category
B2    913
A1    913
B1    913
A2    901
C1    836
Name: count, dtype: int64

In [21]:
# Save the training and validation DataFrames to separate CSV files
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)


In [22]:
train_df.shape

(17904, 2)

In [23]:
val_df.shape

(4476, 2)